In [1]:
import time
import pyspark
import numpy as np
import pandas as pd
import itertools as it
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [2]:
try:
    sc.stop()
except Exception as e:
    print(e)
try:
    sessn.stop()
except Exception as e:
    print(e)
try:
    sc.SparkContext.stop()
except Exception as e:
    print(e)

name 'sessn' is not defined
'SparkContext' object has no attribute 'SparkContext'


In [5]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '8g'), 
                                   ('spark.executor.instances', '10'),
                                   ('spark.executor.cores', '4'), 
                                   ('spark.driver.cores','4'), 
                                   ('spark.driver.memory','8g'),
                                   ('spark.yarn.queue', 'araadh_q1.arapi_sq1'),
                                  ('spark.yarn.executor.memoryOverhead', '1g')])
sessn = SparkSession.builder.appName("HPP").config(conf=conf).enableHiveSupport().getOrCreate().newSession()
sessn

#### Reading the Data

In [6]:
# Input Tables
m_table = "pi_usrs.hpp_ps_modelling_data"
s_table = "pi_usrs.hpp_ps_signal_data_scoring"
# Output Tables
mean_table = "pi_usrs.hpp_ps_mean_data"
icov_table = "pi_usrs.hpp_ps_icov_data"

In [7]:
signal_list =  ['billed_amt_sum',
                 'clm_ln_cnt',
                 'clm_unit_billed',
                 'proc_code_unusual',
                 'pos_unsual',
                 'em_proc_ind',
                 'mod91_ind',
                 'modTC_ind',
                 'mod59_ind',
                 'high_cst_proc_ind',
                 'emgncy_ind']

In [8]:
modelling_data = sessn.table(m_table)
scoring_data = sessn.table(s_table)

In [9]:
scoring_data = scoring_data.select(['servicing_npi', 'srk_clm_id_rj'] + signal_list)
modelling_data = modelling_data.select(['servicing_npi'] + signal_list)

#### Getting the eligible NPIs for scoring

In [10]:
clm_th =10
org_scoring_npi_cnt = scoring_data.select("servicing_npi").distinct().count()
# Generating list Good NPIs (Having more than 10 Claims in Modelling data)
npi_clm_cnt = modelling_data.groupby("servicing_npi").count()
good_npi_clm = npi_clm_cnt.filter(F.col("count")>=clm_th)
good_npi = good_npi_clm.select("servicing_npi")
# data_to_work_scoring -> Consist scoring data for scorable NPIs
data_to_work_scoring = scoring_data.join(good_npi, on = "servicing_npi", how = "inner")
scorable_npi = data_to_work_scoring.select("servicing_npi").distinct()
scorable_npi_cnt = scorable_npi.count()
# data_to_work_modelling -> Consist modelling data for scorable NPIs
data_to_work_modelling = modelling_data.join(scorable_npi, on ="servicing_npi", how ="inner")
print("{} NPI dropped out of {}, {} NPI will be score".format(org_scoring_npi_cnt -scorable_npi_cnt, org_scoring_npi_cnt, scorable_npi_cnt))

257 NPI dropped out of 2028, 1771 NPI will be score


In [11]:
st = time.time()

#### Generating Distribution

In [12]:
scnt = len(signal_list)

In [13]:
signal_comb = list(it.product(signal_list, repeat =2 ))

In [14]:
len(signal_comb)

121

In [15]:
covarince_op = data_to_work_modelling.groupby("servicing_npi").agg(*[F.covar_samp(f[0],f[1]) for f in signal_comb])
mean_op = data_to_work_modelling.groupBy("servicing_npi").agg(*[F.mean(f) for f in signal_list])
covarince_op_pd = covarince_op.toPandas()
mean_op_pd = mean_op.toPandas()

##### Sanity checks for Covariance and Mean

In [16]:
covarince_op_pd = covarince_op_pd.round(3)

In [16]:
modelling_data_pd = data_to_work_modelling.toPandas()

In [17]:
npi_list = modelling_data_pd['servicing_npi'].unique()

In [18]:
test_op =[]
for npi in npi_list:
    sprk_op = covarince_op_pd[covarince_op_pd['servicing_npi'] == npi].iloc[:,1:].values.reshape(scnt, scnt)
    npi_data = modelling_data_pd[modelling_data_pd["servicing_npi"] == npi]
    py_op = npi_data[signal_list].cov().round(3).values
    result = (npi, (sprk_op == py_op).all())
    test_op.append(result)

In [19]:
test_op_df = pd.DataFrame(test_op)
failed_npi = test_op_df[test_op_df[1]==False][0]
test_op_df[test_op_df[1]==False]

0      1
509   1902072945  False
916   1295807543  False
1225  1326049321  False
1279  1437467776  False
1371  1326113564  False

In [20]:
for npi in failed_npi:
    sprk_op = covarince_op_pd[covarince_op_pd['servicing_npi'] == npi].iloc[:,1:].values.reshape(scnt, scnt)
    npi_data = modelling_data_pd[modelling_data_pd["servicing_npi"] == npi]
    py_op = npi_data[signal_list].cov().round(3).values
    matcher = (py_op == sprk_op)
    flse_index = np.where(matcher == False)
    print("{} element value are not same for {}".format(len(flse_index), npi))
    for e in flse_index:
        if len(e) == 2:
            e = tuple(e)
            print("python op {}, spark op {}".format(py_op[e], sprk_op[e]))

2 element value are not same for 1902072945
2 element value are not same for 1295807543
python op 2.362, spark op 2.363
python op 2.362, spark op 2.363
2 element value are not same for 1326049321
python op 0.278, spark op 0.277
python op 0.278, spark op 0.277
2 element value are not same for 1437467776
python op -0.158, spark op -0.157
python op -0.158, spark op -0.157
2 element value are not same for 1326113564


In [21]:
py_op_pd= modelling_data_pd.groupby("servicing_npi").apply(lambda x : x[signal_list].mean()).reset_index()

In [22]:
py_op = py_op_pd.sort_values(by = "servicing_npi").round(3)
sprk_op = mean_op_pd.sort_values(by = "servicing_npi").round(3)
py_op = py_op.iloc[:,1:].values
sprk_op = sprk_op.iloc[:, 1:].values
matcher = (py_op == sprk_op).any()
print(matcher)

True


#### Inverse of Covariance Matrix

In [17]:
covar_dump = covarince_op_pd.values[:,1:]
covar_dump = covar_dump.reshape(scorable_npi_cnt, scnt, scnt)
covar_npi = covarince_op_pd.values[:,0]
col_list = list(covarince_op_pd.head(1).iloc[:,1:])

In [18]:
final_list = []
np.random.seed(20)
randarr = 0.001*np.random.rand(scnt,scnt)
for npi in range(scorable_npi_cnt):
    # Adding noise to the covarinace matrix in order to avoid singular matrix 
    temp_icov = np.linalg.inv((covar_dump[npi] + randarr).astype(float))
    temp_icov = temp_icov.flatten().tolist() 
    final_list.append(temp_icov)

In [19]:
icovariance_op_pd = pd.DataFrame(final_list, columns = col_list)
icovariance_op_pd['servicing_npi'] = covarince_op_pd['servicing_npi']
icovariance_op_pd = icovariance_op_pd[list(covarince_op_pd)]

In [20]:
icovariance_op = sessn.createDataFrame(icovariance_op_pd)

#### Saving the Output

In [ ]:
for s in signal_list:
    s_col_ = "avg({})".format(s)
    s_col_op = "avg_{}".format(s)
    mean_op = mean_op.withColumnRenamed(s_col_, s_col_op)
for s_c in signal_comb:
    s_col_ = "covar_samp({}, {})".format(s_c[0], s_c[1])
    s_col_op = "covar_samp_{}_{}".format(s_c[0], s_c[1])
    icovariance_op = icovariance_op.withColumnRenamed(s_col_, s_col_op)

In [ ]:
mean_op.write.mode("overwrite").format("orc").saveAsTable(mean_table)
icovariance_op.write.mode("overwrite").format("orc").saveAsTable(icov_table)